In [2]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
%matplotlib inline

0.23.2


In [7]:
# 刪除na
df_train = pd.read_csv("Titanic_train.csv")
df_train= df_train.replace('null',np.NaN)
df_train.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [11]:
# Age用平均值補
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
# 其餘刪除
df_train = df_train.dropna()
df_train.isnull().any()

<ipython-input-11-2487ee0c787f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())


PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool

In [23]:
# Q1：產生一個新的變數 (Age_above65_)  Age ≧ 65 為 'Y'，其餘為 'N'。  
def Age_above65_(x):
    if (x>=65):
        return('Y')
    else:
        return('N')
df_train['Age_above65'] = df_train['Age'].apply(Age_above65_)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202 entries, 1 to 889
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  202 non-null    int64  
 1   Survived     202 non-null    int64  
 2   Pclass       202 non-null    int64  
 3   Name         202 non-null    object 
 4   Sex          202 non-null    object 
 5   Age          202 non-null    float64
 6   SibSp        202 non-null    int64  
 7   Parch        202 non-null    int64  
 8   Ticket       202 non-null    object 
 9   Fare         202 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     202 non-null    object 
 12  Age_above65  202 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 22.1+ KB


In [35]:
# Q2：將性別 (sex) 一併列入考慮，產生一個新的變數(Age_above65_female)，當 sex = female 或Age>=65為'Y'，其餘為'N'。
dummy = pd.get_dummies(df_train['Sex'])
df_train = pd.concat([df_train,dummy],axis = 1)
def Age_above65_female(x):
    if [(x>=65) | (x == 1)]:
        return('Y')
    else:
        return('N')
df_train['Age_above65_female'] = df_train[['Age']].apply(Age_above65_female)
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,male,female,male,Age_above65_female,female,male,female,male,female,male
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,1,0,NaN,1,0,1,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,0,1,0,NaN,1,0,1,0,1,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,...,1,0,1,NaN,0,1,0,1,0,1
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,...,0,1,0,NaN,1,0,1,0,1,0
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,...,0,1,0,NaN,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,...,0,1,0,NaN,1,0,1,0,1,0
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,...,1,0,1,NaN,0,1,0,1,0,1
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,...,0,1,0,NaN,1,0,1,0,1,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,...,0,1,0,NaN,1,0,1,0,1,0


In [39]:
# Q3：透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate) 的相關性較高？
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
complete_data=df_train[['Age','Survived_cate','Sex']].dropna()
# age v.s survived
import pingouin as pg
aov = pg.anova(dv='Age', between='Survived_cate', data=complete_data, detailed=True)
etaSq = aov.SS[0] / (aov.SS[0] + aov.SS[1])
def judgment_etaSq(etaSq):
    if etaSq < .01:
        qual = 'Negligible'
    elif etaSq < .06:
        qual = 'Small'
    elif etaSq < .14:
        qual = 'Medium'
    else:
        qual = 'Large'
    return(qual)
judgment_etaSq(etaSq)
# 由結果可知年齡與生存相關性低

'Small'

In [44]:
# sex v.s survived
contTable = pd.crosstab(complete_data['Sex'], complete_data['Survived_cate'])
df = min(contTable.shape[0], contTable.shape[1]) - 1 # degree of freedom
df
import researchpy   
crosstab, res = researchpy.crosstab(complete_data['Sex'], complete_data['Survived_cate'], test='chi-square')
print(res)
def judgment_CramerV(df,V):
# V = Cramer's phi
    if df == 1:
        if V < 0.10:
            qual = 'negligible'
        elif V < 0.30:
            qual = 'small'
        elif V < 0.50:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 2:
        if V < 0.07:
            qual = 'negligible'
        elif V < 0.21:
            qual = 'small'
        elif V < 0.35:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 3:
        if V < 0.06:
            qual = 'negligible'
        elif V < 0.17:
            qual = 'small'
        elif V < 0.29:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 4:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.15:
            qual = 'small'
        elif V < 0.25:
            qual = 'medium'
        else:
            qual = 'large'
    else:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.13:
            qual = 'small'
        elif V < 0.22:
            qual = 'medium'
        else:
            qual = 'large'
    return(qual)

judgment_CramerV(df,res.loc[2,'results'])
# 由結果可之性別與生存有較大的關係

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =   60.0645
1                    p-value =    0.0000
2               Cramer's phi =    0.5453


'large'